In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi
# fugi = reload(fugi)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from itertools import product
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

###550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root");en="550eV";CH="CH11";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

#Select between CH9 and CH11
path = "angular_distr_el/"+CH+"/"

#move first the ID you need
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

loc=path+IDs[0]+dirs[0]

#load theoretical data
pathth = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV';en="KE=11.5eV";

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(pathth + "/**.dat"))

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    allnames.append(temp_name)
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
cosphi_th_adj = np.around(np.array(list(itertools.product(np.linspace(1.,-1.,6).tolist(),np.linspace(-180,180,12).tolist()))),3)
cosphi_th_adj_exp = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
#Loading of the experimental MFPADS
MFPAD_RCR,ctheta_RCR,ctheta_RCR_err,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL,ctheta_RCL_err=fugi.import_MFPAD(fileRCL, loc)
MFPAD_SCR,ctheta_SCR,ctheta_SCR_err=fugi.import_MFPAD(fileSCR, loc)
MFPAD_SCL,ctheta_SCL,ctheta_SCL_err=fugi.import_MFPAD(fileSCL, loc)

MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, theory=False, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, theory=False, a=2)
MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, theory=False, a=2)
MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, theory=False, a=2)

MFPAD_RCR_norm=fugi.normalise_with_err(MFPAD_RCR_phi,normtype=3)
MFPAD_RCL_norm=fugi.normalise_with_err(MFPAD_RCL_phi,normtype=3)
MFPAD_SCR_norm=fugi.normalise_with_err(MFPAD_SCR_phi,normtype=3)
MFPAD_SCL_norm=fugi.normalise_with_err(MFPAD_SCL_phi,normtype=3)

#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
ctheta_axis_red=ctheta_axis_red[0]; #Ffrom ccos(theta)

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2 #phi = (36,), MFPAD_axis = (37,)
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2 #COS = (18,), MFPAD_axis = (19,)
phiMM, cosMM = np.meshgrid(phiM, cosM)

phicos_phi=list(product(phiM, cosM))
phi_full = np.array([el[0] for el in phicos_phi])
cos_full = np.array([el[1] for el in phicos_phi])

phi_rad= phi_full*np.pi/180.
theta_rad= np.arccos(cos_full)

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("len phi_rad = ", phi_rad.shape)
print("len theta_rad = ", theta_rad.shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(axes.ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCL_norm[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(axes.ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCR_norm[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

a=np.ones([6,4])
new=[]
for i,el in enumerate(a):
    for j,el2 in enumerate(el):
        new.append(el2*(i+1)*(j+1))

a_new=np.array(new).reshape(6,4)
print(a_new)
print(a_new.reshape(4,6))

a=np.array([[[1],[2],[3],[4],[5],[6]],[[7],[8],[9],[10],[11],[12]]])
a_test=np.array([[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12]])
a_test_1=a_test+0.1
print(a_test.shape)
fig, axes = plt.subplots(6,2)
for i,ax in zip(range(72),np.flip(axes.T,axis=1).ravel()):
    ax.scatter(a_test[i],i+1,s=100)
    # ax.scatter(a_test_1[i],i+1,s=100, c="r")


In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in phi because of the structure of reshape, do the cos just as a check
temp_RCLCR_phi=np.empty([12,6,648])
temp_SCLCR_phi=np.empty([12,6,648])
temp_totordered_phi=np.empty([12,6,648])
temp_totordered_phi1=np.empty([12,6,648])
# temp_totordered_phi2=np.empty([12,6,648])
# temp_totordered_phi3=np.empty([12,6,648])

nphi=MFPAD_RCL.shape[1];ncos=MFPAD_RCL.shape[2]

#sum helicities: -cos(theta), phi+180 in photon coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            temp_RCLCR_phi[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[j-6,5-i])
            temp_SCLCR_phi[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[j-6,5-i])

        else:
            temp_RCLCR_phi[j,i]=np.add(MFPAD_RCL_phi.reshape(12,6,-1)[j,i],MFPAD_RCR_phi.reshape(12,6,-1)[6+j,5-i])
            temp_SCLCR_phi[j,i]=np.add(MFPAD_SCL_phi.reshape(12,6,-1)[j,i],MFPAD_SCR_phi.reshape(12,6,-1)[6+j,5-i])

#now they are scanned along phi
MFPAD_RCLCR_phi=np.array(temp_RCLCR_phi).reshape(72,nphi,ncos)
MFPAD_SCLCR_phi=np.array(temp_SCLCR_phi).reshape(72,nphi,ncos)
MFPAD_RCLCR_phi_norm=fugi.normalise_with_err(MFPAD_RCLCR_phi,normtype=3)
MFPAD_SCLCR_phi_norm=fugi.normalise_with_err(MFPAD_SCLCR_phi,normtype=3)

MFPAD_SCLCR_phi=fugi.normalise_with_err(MFPAD_SCLCR_phi,normtype=3)

MFPAD_RCLCR_sum_phi=np.sum(MFPAD_RCLCR_phi, axis=0)
MFPAD_SCLCR_sum_phi=np.sum(MFPAD_SCLCR_phi, axis=0)

#sorting
# MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=1)
# MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, theory=False, a=2) #equal to the original!
# MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=1)
# MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, theory=False, a=2) #equal to the original!
    
temp1=[]
for el in MFPAD_RCLCR_phi:
    temp1.append(np.flip(el,axis=1))
MFPAD_RCLCR_phi_flip=np.array(temp1).reshape(72,nphi,ncos)

tempf=[]
for el in temp_RCLCR_phi:
    tempf.append(np.flip(el,axis=1))
temp_RCLCR_phi_flip=np.array(tempf).reshape(12,6,-1)

#sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates
for j in range(12):
    for i in range(6):
        temp_totordered_phi1[j,i]=np.add(temp_SCLCR_phi[j,i],temp_RCLCR_phi_flip[j,5-i])
        temp_totordered_phi[j,i]=np.add(temp_SCLCR_phi[j,i],temp_RCLCR_phi_flip[j,i])
        #temp_totordered_phi2.append(np.add(MFPAD_SCLCR_phi.reshape(6,12,nphi,ncos)[i,j],MFPAD_RCLCR_phi.reshape(6,12,nphi,ncos)[5-i,j]))
        #temp_totordered_phi3.append(np.add(MFPAD_SCLCR_phi.reshape(6,12,nphi,ncos)[i,j],MFPAD_RCLCR_phi.reshape(6,12,nphi,ncos)[i,j]))

MFPAD_totordered_phi=np.array(temp_totordered_phi).reshape(72,nphi,ncos)
MFPAD_totordered_phi1=np.array(temp_totordered_phi1).reshape(72,nphi,ncos)
# MFPAD_totordered_phi2=np.array(temp_totordered_phi2).reshape(72,nphi,ncos)
# MFPAD_totordered_phi3=np.array(temp_totordered_phi3).reshape(72,nphi,ncos)

MFPAD_totordered_phi_norm=fugi.normalise_with_err(MFPAD_totordered_phi,normtype=3)
MFPAD_totordered_phi_norm1=fugi.normalise_with_err(MFPAD_totordered_phi1,normtype=3)
# MFPAD_totordered_phi_norm2=fugi.normalise_with_err(MFPAD_totordered_phi2,normtype=3)
# MFPAD_totordered_phi_norm3=fugi.normalise_with_err(MFPAD_totordered_phi3,normtype=3)

#sorting
# MFPAD_totordered_cos=fugi.sorting_array(MFPAD_totordered, theory=False, a=1)
# MFPAD_totordered_phi=fugi.sorting_array(MFPAD_totordered, theory=False, a=2) #equal to the original!

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_SCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_RCLCR_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
#SUM of S enantiomers two different helicities EXPERIMENTAL
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_totordered_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
#pick the right theoretical MFPAD and inver the cos(theta)_el axis
#NOTE here the cos(theta)_el is sorted in the range [-1:-1]

counts_temp_cosord=[]
for group in frame_srt:
    counts_temp_cosord.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["value"].values)
    if group[0] == "1_1":
        phi_full_sorted=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["phi"].values)
        ctheta_full_sorted=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["cos(theta)"].values)

counts_temp_cosord=np.array(counts_temp_cosord).reshape(72,200,100)

#NOTE these will be sorted according to cos(theta)_el [-1:1]
phiM_th=np.unique(phi_full_sorted)
cosM_th=np.unique(ctheta_full_sorted)
xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, reduced=False, source=True)


xgo_cos_flip,ygo_cos_flip,cosphi_adj_cos_flip = fugi.create_gocoords(a=2, reduced=False, source=True)
dfind=pd.DataFrame(cosphi_adj_cos_flip, columns=["ctheta","phi"])
dfind["item"]=allnames
cosphi_th_std=(dfind.sort_values(by=["ctheta", "phi"], ascending=[False,True]))["item"].values

#NOTE these will be sorted according to cos(theta) or phi PHOTON
# MFPAD_Sth_cos=fugi.sorting_array(counts_temp_cosord, theory=True, items=allnames, a=1)

#MFPAD_S_phi: in photon_coordiantes as allnames 
# OUTPUT a=2 : 6_1', '5_1', '4_1', '3_1', '2_1', '1_1', '6_2', '5_2'
MFPAD_Sth_phi=fugi.sorting_array(counts_temp_cosord, theory=True, items=allnames, a=2)

print("MFPAD th shape ", MFPAD_Sth_phi.shape)
print("phi_full_sorted shape ", phi_full_sorted.shape)


In [ ]:
#SUM of S enantiomers two different helicities THEORETICAL
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Sth_phi[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(axes.ravel()):
    cs,ax=fugi.plot_interpolation(phiM_th,cosM_th,MFPAD_Sth_cos[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
#down sample theoretical MFPAD to 18 36
from scipy.interpolate import griddata

temp=[]
for el in MFPAD_Sth_phi:
    temp.append(griddata(list(zip(phi_full_sorted.reshape(-1),ctheta_full_sorted.reshape(-1))), el.reshape(-1), (phiMM.T, cosMM.T), method="linear"))

MFPAD_Sth_phi_red=np.array(temp).reshape(72,36,18)

In [ ]:
#SUM of S enantiomers two different helicities THEORETICAL red
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_Sth_phi_red[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj_phi[i].T, ha="center", va="center", color="red",fontsize=12)
fugi.overlaygraph(fig)

In [ ]:
MFPAD_check=71
plt.imshow(MFPAD_Sth_phi_red[MFPAD_check].T, cmap="viridis")

In [ ]:
plt.imshow(MFPAD_SCR_norm[MFPAD_check].T, cmap="viridis")


In [ ]:
plt.imshow(MFPAD_RCLCR_phi[MFPAD_check].T, cmap="viridis")

In [ ]:
plt.imshow(MFPAD_SCLCR_phi[MFPAD_check].T, cmap="viridis")

In [ ]:
plt.imshow(MFPAD_totordered_phi[MFPAD_check].T, cmap="viridis")

In [ ]:
plt.imshow(MFPAD_totordered_phi1[MFPAD_check].T, cmap="viridis")

In [ ]:
#describe a set of alpha, beta, gamma angles to rotate the theoretical distribution (would be better to rotate before downsampling)
arr=np.arange(-30,30,4)
#alpha, beta, gamma for 3D rotation
abg=list(product(arr, arr, arr))

a = np.array([el[0] for el in abg])
b = np.array([el[1] for el in abg])
g = np.array([el[2] for el in abg])

In [ ]:
# rotate the theoretical distribution
#for 72 MFPADS the calculation can last for 6-10 minutes for (-30,30,5)
r_temp=[]
n_temp=[]

# these ae now 72 x 1728 MFPAD (36,18)
for el in MFPAD_Sth_phi_red:
    r_temp.append(fugi.rot3d_MFPAD_dist(el,theta_rad,phi_rad,abg,phiMM,cosMM))

MFPAD_Sth_phi_rotated=np.array(r_temp).reshape(72,len(abg),-1)

for el in MFPAD_Sth_phi_rotated:
    n_temp.append(fugi.normalise_with_err(el,normtype=3))

MFPAD_Sth_phi_rotated_norm=np.array(r_temp).reshape(72,len(abg),-1)

new=[]
for el in MFPAD_rotated_norm:
    for el1 in el:
        new.append(np.nan_to_num(el1.reshape(-1),nan=np.average(np.nan_to_num(el1.reshape(-1)))))

MFPAD_rotated_norm=new

In [ ]:
#the min should return the average of the MFPAD, not zero!
print(MFPAD_Sth_phi_rotated_norm[50,MFPAD_check].min())
print(MFPAD_SCLCR_phi_norm[MFPAD_check].max())

#it works but it takes HOURS!!
import matplotlib.animation as ani
fig = plt.figure()

def animate(i=int):
    p=plt.imshow(MFPAD_rotated_norm[20,i].T, cmap="viridis")
    return p

animator = ani.FuncAnimation(fig, animate, frames = len(abg))
animator.save("OUTPUTS/test.mp4", fps =30, writer="ffmpeg")
plt.show()

In [ ]:
#difference square looping along all coordiantes: NOTE they should have filled int he same way
#l2 norm
#to be normalized before 
s_temp=[]
# for i,el_exp in enumerate(MFPAD_SCL_norm): #one enantiomer, one helicity
for i,el_exp in enumerate(MFPAD_totordered_phi_norm): #all together
    d_temp=[]
    for el_th in MFPAD_Sth_phi_rotated_norm[i]:
        # d_temp.append(np.sum((el_exp.reshape(-1) - el_th.reshape(-1)))**2)
        d_temp.append(np.sum((el_exp.reshape(-1)-el_th.reshape(-1))**2)) #len(d_temp) = len(abg)
    s_temp.append(np.sqrt(d_temp)) #len(s_temp) =len(d_temp)*72

d_all=np.array(s_temp).reshape(72,len(abg))
d_sum=np.sum(d_all, axis=0)
d_norm=fugi.normalise_with_err(np.array(d_sum),normtype=3)

print("d_temp length = ", len(d_temp))
print("s_temp shaoe = ", np.array(s_temp).shape)
print("d_all shape = ", d_all.shape)
print("d_sum shape = ", d_sum.shape)

In [ ]:
d_tot=np.column_stack((a,b,g,d_norm)).reshape(-1,4)
d_min=d_norm.min()
output=[]
for el in d_tot:
    if el[3] <= d_min*1.0002:
        output.append(el)
print("For the collective 72 MFPADs")
sorted(output, key = lambda x:x[3])

In [ ]:
d_temp_s=[]
for el_th in MFPAD_Sth_phi_rotated_norm[71]:
    d_temp_s.append(np.sum((MFPAD_SCL_norm[71].reshape(-1)-el_th.reshape(-1))**2)) #len(d_temp) = len(abg)
    # d_temp_s.append(np.sum((MFPAD_totordered_phi_norm[71].reshape(-1)-el_th.reshape(-1))**2)) #len(d_temp) = len(abg)

d_all_s=np.sqrt(np.array(d_temp_s))
d_norm_s=fugi.normalise_with_err(np.array(d_all_s),normtype=3)

print("d_temp_s length = ", len(d_temp_s))
print("d_norm_s shape = ", d_norm_s.shape)

In [ ]:
print("For the S enantiomer MFPADs")
d_tot_s=np.column_stack((a,b,g,d_norm_s)).reshape(-1,4)
d_min_s=d_norm_s.min()
output_s=[]
for el in d_tot_s:
    if el[3] <= d_min_s*1.0002:
        output_s.append(el)
sorted(output_s, key = lambda x:x[3])

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')

delta=d_norm.max()-d_norm.min()

d_masked = np.ma.masked_array(d_norm, d_norm>=d_norm.min()+delta*0.5)
ax.scatter(a, b, g, c=d_masked, alpha=0.2, s=10, cmap=plt.hot())

d_masked = np.ma.masked_array(d_norm, d_norm>=d_norm.min()+delta*0.3)
img = ax.scatter(a, b, g, c=d_masked, alpha=0.5, s=50, cmap=plt.hot())

d_masked = np.ma.masked_array(d_norm, d_norm>=d_norm.min()+delta*0.05)
ax.scatter(a, b, g, c=d_masked, alpha=0.8, s=250, cmap=plt.hot())
# ax.scatter(a, b, c=d_masked, zs=-90, zdir='z', s=60, cmap=plt.hot())
# ax.scatter(a, g, c=d_masked, zs=-70, zdir='y', s=60, cmap=plt.hot())
# ax.scatter(b, g, c=d_masked, zs=70,  zdir='x', s=60, cmap=plt.hot())

ax.set_xlabel('x alpha [DEG]', labelpad=25)
ax.set_ylabel('y beta [DEG]', labelpad=25)
ax.set_zlabel('z gamma [DEG]', labelpad=25)

ax.view_init(elev=30., azim=180.)

fig.colorbar(img)
plt.show()

In [ ]:
fig1 = go.Figure(data=[go.Scatter3d(x=a, y=b, z=g, mode='markers',
                    marker=dict( size=5,color=d_norm, colorscale='temps', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

# fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

In [ ]:
fig1 = go.Figure(data=[go.Scatter3d(x=a, y=b, z=g, mode='markers',
                    marker=dict( size=5,color=d_norm_s, colorscale='temps', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

# fig1.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig1.show()

In [ ]:
def get_the_slice(x,y,z, surfacecolor):
    return go.Surface(x=x,
                      y=y,
                      z=z,
                      surfacecolor=surfacecolor,
                      coloraxis='coloraxis')



def get_lims_colors(surfacecolor):# color limits for a slice
    return np.min(surfacecolor), np.max(surfacecolor)

sminz, smaxz = get_lims_colors(d_norm_s)
slice_z = get_the_slice(a, b, g, d_norm_s)

fig1 = go.Figure(data=[slice_z])
fig1.update_layout(
         title_text='Slices in volumetric data', 
         title_x=0.5,
         width=700,
         height=700,
         scene_zaxis_range=[-2,2], 
         coloraxis=dict(colorscale='BrBG',
                        colorbar_thickness=25,
                        colorbar_len=0.75)) 